In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from fpdf import FPDF
import os
import numpy as np
from datetime import datetime

# Function to create a professional PDF report
def create_report(kpis, product_metrics, customer_metrics, country_sales, monthly_sales):
    class PDF(FPDF):
        def header(self):
            # Logo
            self.image('northwind_logo.png', 10, 8, 33)  # Placeholder for logo
            # Arial bold 15
            self.set_font('Arial', 'B', 15)
            # Move to the right
            self.cell(80)
            # Title
            self.cell(30, 10, 'Northwind Traders - Data Analysis Report', 0, 0, 'C')
            # Line break
            self.ln(20)

        def footer(self):
            # Position at 1.5 cm from bottom
            self.set_y(-15)
            # Arial italic 8
            self.set_font('Arial', 'I', 8)
            # Page number
            self.cell(0, 10, 'Page ' + str(self.page_no()) + '/{nb}', 0, 0, 'C')
            # Date
            self.cell(-40, 10, datetime.now().strftime('%Y-%m-%d'), 0, 0, 'R')

    # Create PDF instance
    pdf = PDF()
    pdf.alias_nb_pages()
    pdf.add_page()

    # Executive Summary
    pdf.set_font('Arial', 'B', 16)
    pdf.cell(0, 10, 'Executive Summary', 0, 1)
    pdf.set_font('Arial', '', 12)
    pdf.multi_cell(0, 10, 'This report provides a comprehensive analysis of Northwind Traders business performance. Key insights are presented to inform strategic decisions aimed at increasing average ticket value and reducing customer churn.')

    # Key Performance Indicators
    pdf.set_font('Arial', 'B', 16)
    pdf.cell(0, 10, 'Key Performance Indicators', 0, 1, 'L')
    pdf.set_font('Arial', 'B', 12)

    # Total Revenue
    pdf.cell(60, 10, 'Total Revenue:', 0, 0)
    pdf.set_font('Arial', '', 12)
    pdf.cell(0, 10, f"${kpis['total_revenue']:,.2f}", 0, 1)

    # Average Order Value
    pdf.set_font('Arial', 'B', 12)
    pdf.cell(60, 10, 'Average Order Value:', 0, 0)
    pdf.set_font('Arial', '', 12)
    pdf.cell(0, 10, f"${kpis['avg_order_value']:,.2f}", 0, 1)

    # CLV
    pdf.set_font('Arial', 'B', 12)
    pdf.cell(60, 10, 'Customer Lifetime Value:', 0, 0)
    pdf.set_font('Arial', '', 12)
    pdf.cell(0, 10, f"${kpis['customer_lifetime_value']:,.2f}", 0, 1)

    # Churn Rate
    pdf.set_font('Arial', 'B', 12)
    pdf.cell(60, 10, 'Potential Churn Rate:', 0, 0)
    pdf.set_font('Arial', '', 12)
    pdf.cell(0, 10, f"{kpis['churn_rate']*100:.2f}%", 0, 1)

    # Graphs
    pdf.add_page()
    pdf.set_font('Arial', 'B', 16)
    pdf.cell(0, 10, 'Sales Analysis', 0, 1)

    # Add the sales trend chart
    pdf.image('northwind_analysis.png', x=10, y=30, w=190)
    pdf.ln(140)  # Move down to avoid overlap

    # AOV Trends
    pdf.image('northwind_aov_trend.png', x=10, y=170, w=190)

    # Product Analysis
    pdf.add_page()
    pdf.set_font('Arial', 'B', 16)
    pdf.cell(0, 10, 'Product Analysis', 0, 1)
    pdf.set_font('Arial', '', 12)
    pdf.multi_cell(0, 10, 'Analysis of top performing products and opportunities for increasing average ticket value.')

    # Add a table of top 5 products
    pdf.set_font('Arial', 'B', 12)
    top_products = product_metrics.sort_values('total_revenue', ascending=False).head(5)

    # Table header
    pdf.ln(5)
    pdf.cell(80, 10, 'Product Name', 1, 0, 'C')
    pdf.cell(40, 10, 'Total Revenue', 1, 0, 'C')
    pdf.cell(40, 10, 'Quantity Sold', 1, 0, 'C')
    pdf.cell(30, 10, 'Avg Discount', 1, 1, 'C')

    # Table data
    pdf.set_font('Arial', '', 11)
    for _, row in top_products.iterrows():
        pdf.cell(80, 10, str(row['product_name'])[:35], 1, 0)
        pdf.cell(40, 10, f"${row['total_revenue']:,.2f}", 1, 0)
        pdf.cell(40, 10, f"{row['total_quantity']:,}", 1, 0)
        pdf.cell(30, 10, f"{row['avg_discount']*100:.1f}%", 1, 1)

    # Customer Analysis
    pdf.add_page()
    pdf.set_font('Arial', 'B', 16)
    pdf.cell(0, 10, 'Customer Analysis & Churn Risk', 0, 1)
    pdf.set_font('Arial', '', 12)
    pdf.multi_cell(0, 10, 'Analysis of customer behavior and identification of churn risk factors.')

    # Add a table of customers with high churn risk
    pdf.set_font('Arial', 'B', 12)
    churn_risk = customer_metrics[customer_metrics['days_since_last_order'] > 90].sort_values('total_revenue', ascending=False).head(5)

    # Table header
    pdf.ln(5)
    pdf.cell(65, 10, 'Company', 1, 0, 'C')
    pdf.cell(30, 10, 'Country', 1, 0, 'C')
    pdf.cell(30, 10, 'Revenue', 1, 0, 'C')
    pdf.cell(30, 10, 'Days Inactive', 1, 0, 'C')
    pdf.cell(35, 10, 'Lifetime (days)', 1, 1, 'C')

    # Table data
    pdf.set_font('Arial', '', 11)
    for _, row in churn_risk.iterrows():
        pdf.cell(65, 10, str(row['company_name'])[:30], 1, 0)
        pdf.cell(30, 10, str(row['country']), 1, 0)
        pdf.cell(30, 10, f"${row['total_revenue']:,.2f}", 1, 0)
        pdf.cell(30, 10, f"{row['days_since_last_order']}", 1, 0)
        pdf.cell(35, 10, f"{row['customer_lifetime_days']}", 1, 1)

    # Geographic Analysis
    pdf.add_page()
    pdf.set_font('Arial', 'B', 16)
    pdf.cell(0, 10, 'Geographic Sales Distribution', 0, 1)

    # Table of top countries by sales
    pdf.set_font('Arial', 'B', 12)
    top_countries = country_sales.head(10)

    # Table header
    pdf.ln(5)
    pdf.cell(60, 10, 'Country', 1, 0, 'C')
    pdf.cell(45, 10, 'Total Revenue', 1, 0, 'C')
    pdf.cell(45, 10, 'Customer Count', 1, 0, 'C')
    pdf.cell(40, 10, 'Avg Rev/Customer', 1, 1, 'C')

    # Table data
    pdf.set_font('Arial', '', 11)
    for _, row in top_countries.iterrows():
        pdf.cell(60, 10, str(row['country']), 1, 0)
        pdf.cell(45, 10, f"${row['total_revenue']:,.2f}", 1, 0)
        pdf.cell(45, 10, f"{row['customer_count']}", 1, 0)
        pdf.cell(40, 10, f"${row['avg_revenue_per_customer']:,.2f}", 1, 1)

    # Recommendations
    pdf.add_page()
    pdf.set_font('Arial', 'B', 16)
    pdf.cell(0, 10, 'Recommendations', 0, 1)
    pdf.set_font('Arial', '', 12)

    # Increasing Average Ticket Value
    pdf.set_font('Arial', 'B', 14)
    pdf.cell(0, 10, '1. Strategies to Increase Average Ticket Value:', 0, 1)
    pdf.set_font('Arial', '', 12)
    pdf.multi_cell(0, 10, '• Implement cross-selling strategies with top-performing products\n'
                         '• Review pricing strategy and discount policies\n'
                         '• Create bundled product offerings\n'
                         '• Targeted promotions for high-margin products')

    # Reducing Churn
    pdf.set_font('Arial', 'B', 14)
    pdf.cell(0, 10, '2. Strategies to Reduce Customer Churn:', 0, 1)
    pdf.set_font('Arial', '', 12)
    pdf.multi_cell(0, 10, '• Implement proactive outreach to customers at risk\n'
                         '• Create loyalty program to reward repeat customers\n'
                         '• Regular check-ins with high-value customers\n'
                         '• Customer feedback program to identify pain points')

    # Implementation Plan
    pdf.set_font('Arial', 'B', 14)
    pdf.cell(0, 10, '3. Implementation Plan for Data-Driven Decision Making:', 0, 1)
    pdf.set_font('Arial', '', 12)
    pdf.multi_cell(0, 10, '• Integrate data from ERP, Salesforce CRM, and ContaAzul\n'
                         '• Implement PowerBI for real-time dashboard access\n'
                         '• Create automated reporting for key metrics\n'
                         '• Implement data governance processes\n'
                         '• Train team members on data interpretation')

    # ROI Projection
    pdf.set_font('Arial', 'B', 14)
    pdf.cell(0, 10, '4. Expected Return on Investment:', 0, 1)
    pdf.set_font('Arial', '', 12)
    pdf.multi_cell(0, 10, '• 10% increase in Average Order Value within 6 months\n'
                         '• 15% reduction in customer churn within 1 year\n'
                         '• Revenue increase of approximately R$2.25M annually\n'
                         '• Estimated ROI of 300% in the first year')

    # Conclusion
    pdf.add_page()
    pdf.set_font('Arial', 'B', 16)
    pdf.cell(0, 10, 'Conclusion', 0, 1)
    pdf.set_font('Arial', '', 12)
    pdf.multi_cell(0, 10, 'This analysis demonstrates significant opportunities for Northwind Traders to improve performance through data-driven decision making. By focusing on increasing average ticket value and reducing customer churn, the company can accelerate growth and improve profitability.\n\n'
                         'The implementation of an integrated data platform will provide management with timely, accurate insights to make strategic decisions and monitor progress toward goals.')

    # Save the PDF
    pdf.output('Northwind_Traders_Analysis_Report.pdf', 'F')

# Try to create the report
try:
    # Placeholder for a logo - you might need to create this
    from PIL import Image, ImageDraw, ImageFont
    img = Image.new('RGB', (300, 100), color = (255, 255, 255))
    d = ImageDraw.Draw(img)
    d.text((10,40), "Northwind Traders", fill=(0,0,0))
    img.save('northwind_logo.png')

    # Create the report
    create_report(kpis, product_metrics, customer_metrics, country_sales, monthly_sales)
    print("Report created successfully as 'Northwind_Traders_Analysis_Report.pdf'")
except Exception as e:
    print(f"Error creating report: {e}")